In [1]:
import vfvlib

In [3]:
sts = vfvlib.ipt_gsheet(1)
sts[0]

'In small towns the same workman makes chairs and doors and tables, and often the same person builds houses'

In [5]:
print(vfvlib.structure_analysis(sts[0]))

<In small towns the same workman makes chairs and doors and tables, and often the same person builds houses.>
- In - preposition - 에
- small - adjective - 작은
- towns - noun - 마을
- the - determiner - 그
- same - adjective - 같은
- workman - noun - 일꾼
- makes - verb - 만들다
- chairs - noun - 의자
- and - conjunction - 그리고
- doors - noun - 문
- and - conjunction - 그리고
- tables - noun - 탁자
- and - conjunction - 그리고
- often - adverb - 자주
- the - determiner - 그
- same - adjective - 같은
- person - noun - 사람
- builds - verb - 짓다
- houses - noun - 집
그 작은 마을에서 같은 일꾼이 의자와 문과 탁자를 만들고, 종종 같은 사람이 집을 짓습니다.
